In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.491, 0.482, 0.446],
            std= [0.247, 0.243, 0.261]
        )]) # TODO: Automate calculation given a dataset

trainset = torchvision.datasets.CIFAR10(root='./raid/data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./raid/data', train=False,
                                       download=True, transform=transform)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

In [31]:
from torchvision import models
import torch.nn as nn
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(in_features=512, out_features=10)

In [32]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

In [ ]:
model.train()
for epoch in range(15):  # loop over the dataset multiple times
    print("########## {} ##########".format(epoch+1))
    train_loss = 0.0
    total  = 0
    correct = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        if torch.cuda.is_available():
            model.cuda()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        if i % 200 == 199:    # print every 200 mini-batches
            print("Loss: {} | Acc: {} | {}/{}".format(train_loss/200, 100.*correct/total, correct, total))
            train_loss = 0

print('Finished Training')

########## 1 ##########
Loss: 2.4020590311288834 | Acc: 13.875 | 111/800
Loss: 2.260924884676933 | Acc: 16.1875 | 259/1600
Loss: 2.1441143172979356 | Acc: 18.333333333333332 | 440/2400
Loss: 2.122776813209057 | Acc: 20.0 | 640/3200
Loss: 2.0903451085090636 | Acc: 20.525 | 821/4000
Loss: 2.047880454957485 | Acc: 21.541666666666668 | 1034/4800
Loss: 2.0258519053459167 | Acc: 22.375 | 1253/5600
Loss: 2.014839940965176 | Acc: 22.71875 | 1454/6400
Loss: 1.985060502886772 | Acc: 23.34722222222222 | 1681/7200
Loss: 1.9067488738894462 | Acc: 24.1125 | 1929/8000
Loss: 1.8706936863064767 | Acc: 24.852272727272727 | 2187/8800
Loss: 1.8785923615098 | Acc: 25.395833333333332 | 2438/9600
Loss: 1.877520509660244 | Acc: 26.0 | 2704/10400
Loss: 1.8314386662840842 | Acc: 26.642857142857142 | 2984/11200
Loss: 1.811381601393223 | Acc: 27.283333333333335 | 3274/12000
Loss: 1.8709281343221664 | Acc: 27.59375 | 3532/12800
Loss: 1.8225640466809272 | Acc: 28.00735294117647 | 3809/13600
Loss: 1.7717475435137748

Loss: 0.7927097563445568 | Acc: 70.78571428571429 | 7928/11200
Loss: 0.9105615282058716 | Acc: 70.81666666666666 | 8498/12000
Loss: 0.8148881486058235 | Acc: 70.90625 | 9076/12800
Loss: 0.9042763666808605 | Acc: 70.69852941176471 | 9615/13600
Loss: 0.8561027243733406 | Acc: 70.67361111111111 | 10177/14400
Loss: 0.823942962884903 | Acc: 70.63815789473684 | 10737/15200
Loss: 0.8020720602571965 | Acc: 70.725 | 11316/16000
Loss: 0.9100853070616722 | Acc: 70.64285714285714 | 11868/16800
Loss: 0.83401825979352 | Acc: 70.58522727272727 | 12423/17600
Loss: 0.8183437976241111 | Acc: 70.68478260869566 | 13006/18400
Loss: 0.844349091053009 | Acc: 70.72395833333333 | 13579/19200
Loss: 0.7919922173023224 | Acc: 70.825 | 14165/20000
Loss: 0.8757037968933582 | Acc: 70.77403846153847 | 14721/20800
Loss: 0.8188944709300995 | Acc: 70.81018518518519 | 15295/21600
Loss: 0.8035459810495377 | Acc: 70.89285714285714 | 15880/22400
Loss: 0.8067472165822983 | Acc: 70.95689655172414 | 16462/23200
Loss: 0.7730268

In [29]:
torch.save(model.state_dict(), "./raid/trained_models/pred_class.pt")